<h1 style="text-align:center;line-height:1.5em;font-size:30px;">River Boundary Detection and River Depth Determination</h1>
<h3 style="text-align:center;line-height:1.5em;font-size:20px;">Improving the 3D Representation of Rivers in Digital Elevation Models (DEM)</h3>
<hr>

<br> 

**Notebook Structure**
<br> 
  
 1. River Boundary Detection
 2. Model Selection <br>
 3. River Depth Determination<br>



<h1> River Boundary Selection</h1><br><p>Automating the detection of river boundaries stands as an imperative step in optimizing workflow processes. Currently, the predominant manual method relies heavily on on-screen digitization to delineate river boundaries, but its inefficiency becomes pronounced, especially in vast expanses. Thus, adopting an automated approach for river boundary detection could notably uplift process efficiency, concurrently preserving precious human resources and time.</p><br>
<p>Furthermore, this automated solution would not only expedite workflow but also facilitate real-time or near-real-time processing. Such capabilities in the algorithms could ensure swift integration of updated river boundaries into the 3D Representation of Rivers in DEM, leading to a dynamic and accurate portrayal of river systems.</p>

<h1> Model Selection</h1>

<h3>U-Net</h3>
<p>U-Net is a deep learning architecture widely used for image segmentation tasks. In river detection, U-Net can be trained on satellite or aerial imagery to learn the features associated with rivers. By inputting an image patch, U-Net produces a corresponding segmentation map where each pixel is classified as either part of the river or not. This enables automatic delineation of river boundaries by analyzing the spatial patterns and textures unique to rivers.</p>
<p> For U-Net training several DEM derived feature layers such as:
    <ul>
  <li>Slope</li>
  <li>Flow accumulation</li>
  <li>Geometric curvature</li>
  <li>Positive openness</li>
  <li>Topographic position index</li>
</ul>
are required to develop the model. Due to computation limitation and time constraint three of the five layers were developed. However, the model training is not completed.
<br>
<h3>SegmentGeo</h3>
<p>SegmentGEo is a geospatial-focused segmentation framework designed to work with satellite imagery and geospatial data. It takes advantage of both spectral and spatial information present in these images. In river detection, SegmentGEo can utilize multispectral data, incorporating bands beyond visible light (such as infrared), which are valuable for distinguishing water bodies like rivers from their surroundings. By combining spectral and spatial cues, SegmentGEo enhances the accuracy of river boundary extraction.<br>
Described below it the implemetation using SegmentGeo.</p>


## Install dependencies

In [ ]:
!%pip install segment-geospatial groundingdino-py leafmap localtileserver

In [ ]:
import leafmap
from samgeo import tms_to_geotiff
from samgeo.text_sam import LangSAM

## Create an interactive map

In [ ]:
m = leafmap.Map(center=[40.683934, -86.714739], zoom=18, height="800px")
# m.add_basemap("SATELLITE")
m

## Download a sample image

Pan and zoom the map to select the area of interest. Use the draw tools to draw a polygon or rectangle on the map

In [ ]:
bbox = m.user_roi_bounds()
if bbox is None:
    bbox = [-51.2565, -22.1777, -51.2512, -22.175]

image = "Image.tif"
tms_to_geotiff(output=image, bbox=bbox, zoom=17, source="Satellite", overwrite=True)

## Display the downloaded image on the map.

In [ ]:
m.layers[-1].visible = False
m.add_raster(image, layer_name="Image")
m

## Initialize LangSAM class

The initialization of the LangSAM class might take a few minutes. The initialization downloads the model weights and sets up the model for inference.

In [ ]:
sam = LangSAM()

## Specify text prompts

Used to limit the output to only rivers


sam.predict(image, text_prompt, box_threshold=0.25, text_threshold=0.24)

### Visualize the results

Show the result with bounding boxes on the map.

In [ ]:
sam.show_anns(
    cmap='Greens',
    box_color='red',
    title='Automatic Segmentation of Rivers',
    blend=True,
)

## Show the result as a grayscale image

In [ ]:
sam.show_anns(
    cmap='Greys_r',
    add_boxes=False,
    alpha=1,
    title='Automatic Segmentation of Rivers',
    blend=False,
    output='river.tif',
)

## Convert the result to a vector format.

In [ ]:
sam.raster_to_vector("river.tif", "river.shp")

## Show the results on the interactive map.

In [ ]:
m.add_raster("river.tif", layer_name="Rivers", palette="Greens", opacity=0.5, nodata=0)
style = {
    "color": "#3388ff",
    "weight": 2,
    "fillColor": "#7c4185",
    "fillOpacity": 0.5,
}
m.add_vector("river.shp", layer_name="Vector", style=style)
m

***

<h1>River Depth Determination</h1>

<p> Automatic river depth determination using satellite imagery offers a significant advancement over manual surveys in terms of efficiency and coverage. Manual surveys are often time-consuming, resource-intensive, and limited by accessibility to various parts of the river and practically impossibe to survey al river bodies. In contrast, satellite imagery provides a broader and more accessible view of rivers, enabling consistent and frequent monitoring of river depth changes.</p><br>

<h3>Methods</h3>
<p>Remote Sensing and Bathymetric Models: Satellite imagery captures water surface patterns, which vary with changes in river depth. Remote sensing techniques, such as using multispectral data, LiDAR, or Synthetic Aperture Radar (SAR), can be employed to measure these changes. Bathymetric models are then created by correlating the observed surface patterns with known river bathymetry. These models allow for river depth estimation over large areas without the need for extensive fieldwork.

Machine Learning Algorithms: Machine learning models, such as neural networks, can be trained on satellite images and corresponding ground-truth river depth data collected through manual surveys. These models learn to recognize features in the imagery that correlate with varying river depths. Once trained, they can predict river depths in new satellite images. This method provides a scalable solution for frequent and wide-ranging river depth assessment.
</p>

<h3> Efforts and Challanges</h3>
1. Bathymetric measurement using ICESAT and Landsat 9<br>
    Missing or unavailabe electronic water chart<br>
2. Estimation of maximum depth (“missing depth”) of river channels(Results in another Notebook)